# JSSP comparação entre euristicas Classicas: codigo fonte

## importação de libs

In [46]:
# %pip install mealpy
# %pip install numpy
%pip install -r ../requirements.txt


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [47]:
import os
import importlib.util
from classes.jssp import jssp
import numpy as np
from classes.jssp import jssp
from mealpy import SA
from mealpy.utils.space import FloatVar
import matplotlib.pyplot as plt
import time
import csv

## funções adicionais

In [48]:
# importar casos de teste
def import_tests_cases(nome_dict : str) -> dict:

    caminho_absoluto = os.path.abspath("../tests/test1.py")

    spec = importlib.util.spec_from_file_location("modulo_temp", caminho_absoluto)
    
    if spec is None or spec.loader is None:
        raise ImportError(f"Não foi possível carregar o módulo do arquivo: {caminho_absoluto}")
    
    modulo = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(modulo)

    return getattr(modulo, nome_dict)

def decode_solution(solution_vec, operations):
    import numpy as np
    order = np.argsort(solution_vec)

    machine_available = {}
    job_operation_count = {}  # Conta quantas operações de cada job já foram executadas
    job_last_end_time = {}    # Último tempo de fim de operação para cada job

    print("Ordem com máquina usada:")

    for i, idx in enumerate(order):
        op = operations[idx]
        job = op["job"]
        
        # Inicializa contadores se necessário
        if job not in job_operation_count:
            job_operation_count[job] = 0
            job_last_end_time[job] = 0

        # Escolher a máquina que fica disponível primeiro
        machine = min(op["machines"], key=lambda m: machine_available.get(m, 0))

        # Considera tanto a disponibilidade da máquina quanto a precedência do job
        machine_ready_time = machine_available.get(machine, 0)
        job_ready_time = job_last_end_time[job]
        start_time = max(machine_ready_time, job_ready_time)
        end_time = start_time + op["duration"]

        # Atualiza os tempos
        machine_available[machine] = end_time
        job_last_end_time[job] = end_time
        job_operation_count[job] += 1

        print(f"{i+1}: {op['job']} Op{job_operation_count[job]}, Máquina {machine}, Duração {op['duration']}, Início {start_time}, Fim {end_time}, Equipamento {op.get('equipments', 'N/A')}")

def make_fitness_function(instance: jssp):
    """
    Cria função fitness que considera operações sequenciais dentro de cada job.
    Operações do mesmo job devem ser executadas em ordem sequencial (Op1 → Op2 → Op3...).
    Aplica penalidade severa se a ordem for violada.
    """
    operations = instance.get_flattened_operations()

    def fitness(solution):
        import numpy as np

        priority_order = np.argsort(solution)

        machine_available = {}    # ex: {1: 5} → máquina 1 estará livre no tempo 5
        job_operation_count = {}  # ex: {"job_1": 2} → job_1 já executou 2 operações
        job_last_end_time = {}    # ex: {"job_1": 4} → última operação do job_1 terminou no tempo 4
        job_next_expected_op = {} # ex: {"job_1": 3} → próxima operação esperada do job_1 é a Op 3
        end_times = []            # armazenará o tempo de término de cada operação
        equipment_in_use = {}     # ex: {"equip_1": 8} → equipamento 1 estará livre no tempo 8
        
        # Contador de violações de precedência
        precedence_violations = 0
        
        # Fator de penalidade (muito alto para desencorajar violações)
        PENALTY_FACTOR = 1000

        for idx in priority_order:
            op = operations[idx]  # Recupera a operação pelo índice

            job = op["job"]               # Nome do job (ex: "job_1")
            machines = op["machines"]     # Lista de máquinas disponíveis
            duration = op["duration"]     # Duração da operação
            equipments = op.get("equipments", [])  # Lista de equipamentos necessários
            operation_id = op.get("operation_id", 1)  # ID da operação dentro do job

            # Inicializa contadores se necessário
            if job not in job_operation_count:
                job_operation_count[job] = 0
                job_last_end_time[job] = 0
                job_next_expected_op[job] = 1  # Primeira operação esperada é sempre ID 1

            # VERIFICAÇÃO DE PRECEDÊNCIA: A operação deve ser executada na ordem correta
            expected_op_id = job_next_expected_op[job]
            
            if operation_id != expected_op_id:
                # VIOLAÇÃO DETECTADA!
                precedence_violations += 1
                print(f"⚠️  VIOLAÇÃO DE PRECEDÊNCIA: Job {job} tentou executar Op {operation_id}, mas esperava Op {expected_op_id}")
                
                # Aplicar penalidade severa mas continuar o cálculo para debug
                # Nota: Ainda processamos a operação para manter consistência do algoritmo
            
            # Atualizar a próxima operação esperada
            if operation_id == expected_op_id:
                job_next_expected_op[job] = operation_id + 1

            # Seleciona a máquina disponível mais cedo
            machine = min(machines, key=lambda m: machine_available.get(m, 0))

            # Calcula o tempo de início considerando:
            # 1. Disponibilidade da máquina
            # 2. Precedência do job (operações sequenciais)
            # 3. Disponibilidade dos equipamentos (se houver)
            machine_ready_time = machine_available.get(machine, 0)
            job_ready_time = job_last_end_time[job]

            # Verifica disponibilidade dos equipamentos
            equipment_ready_time = 0
            if equipments:
                for equipment in equipments:
                    equipment_ready_time = max(equipment_ready_time, equipment_in_use.get(equipment, 0))

            # O tempo de início é o máximo entre todos os tempos de disponibilidade
            start_time = max(machine_ready_time, job_ready_time, equipment_ready_time)
            end_time = start_time + duration

            # Atualiza as disponibilidades
            machine_available[machine] = end_time
            job_last_end_time[job] = end_time
            job_operation_count[job] += 1

            # Atualiza disponibilidade dos equipamentos
            if equipments:
                for equipment in equipments:
                    equipment_in_use[equipment] = end_time

            end_times.append(end_time)  # Salva o tempo final da operação

        # Calcula o makespan base
        makespan = max(end_times) if end_times else 0
        
        # Aplica penalidade por violações de precedência
        penalty = precedence_violations * PENALTY_FACTOR
        
        # Fitness final = makespan + penalidade
        final_fitness = makespan + penalty
        
        # Debug: mostrar quando há violações
        if precedence_violations > 0:
            print(f"🚨 SOLUÇÃO INVÁLIDA: {precedence_violations} violações de precedência")
            print(f"   Makespan base: {makespan:.2f}")
            print(f"   Penalidade: {penalty}")
            print(f"   Fitness final: {final_fitness:.2f}")

        return final_fitness,  # Retorna o fitness com penalidade como tupla

    return fitness

def save_results_to_csv(times, solutions, instance_data, filename="results.csv"):
    """
    Salva os resultados de tempo e soluções em um arquivo CSV.
    
    Parameters:
    times: lista de [tempo_execucao, id_solucao]
    solutions: lista de objetos solution com atributos id, target.fitness, solution
    instance_data: dicionário com os dados da instância (contém timespan)
    filename: nome do arquivo CSV a ser criado
    """
   
    
    # Criar um dicionário para mapear id para tempo de execução
    time_dict = {id_sol: tempo for tempo, id_sol in times}
    
    # Obter o timespan da instância
    timespan = instance_data.get("timespan", "N/A")
    
    # Preparar os dados para o CSV
    csv_data = []
    
    for solution in solutions:
        solution_id = solution.id
        execution_time = time_dict.get(solution_id, "N/A")
        fitness = solution.target.fitness
        
        # Converter o vetor solução para string para facilitar armazenamento
        solution_vector = str(solution.solution.tolist()) if hasattr(solution.solution, 'tolist') else str(solution.solution)
        
        csv_data.append({
            'id': solution_id,
            'execution_time': execution_time,
            'fitness': fitness,
            'timespan': timespan,
            'solution_vector': solution_vector
        })
    
    # Salvar no CSV
    fieldnames = ['id', 'execution_time', 'fitness', 'timespan', 'solution_vector']
    
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(csv_data)
    
    print(f"Resultados salvos em {filename}")
    print(f"Total de {len(csv_data)} soluções salvas")
    print(f"Timespan da instância: {timespan}")
    
    return filename

## Configurações gerais

In [49]:
# dados para os testes iniciais
data = import_tests_cases("best")
instance = jssp(data)
fitness_func = make_fitness_function(instance)
num_ops = len(instance.get_flattened_operations())

print(f"Número de operações: {num_ops}")
print("Operações carregadas:")
operations = instance.get_flattened_operations()
for i, op in enumerate(operations):
    print(f"  {i}: Job {op['job']}, Op {op['operation_id']}, Máquinas {op['machines']}, Duração {op['duration']}")

# Teste com solução que respeita a ordem (boa)
print("\n" + "="*60)
print("TESTE 1: Solução que RESPEITA a ordem das operações")
print("="*60)
good_solution = [0.1, 0.9, 0.2, 0.8]  # job_1: Op1→Op2, job_2: Op1→Op2
fitness_good = fitness_func(good_solution)
print(f"Fitness (ordem correta): {fitness_good[0]}")

# Teste com solução que viola a ordem (ruim)
print("\n" + "="*60)
print("TESTE 2: Solução que VIOLA a ordem das operações")
print("="*60)
bad_solution = [0.9, 0.1, 0.8, 0.2]   # job_1: Op2→Op1, job_2: Op2→Op1
fitness_bad = fitness_func(bad_solution)
print(f"Fitness (ordem violada): {fitness_bad[0]}")

print(f"\nDiferença de fitness: {fitness_bad[0] - fitness_good[0]}")

# Definindo o problema
problem = {
    "obj_func": fitness_func,
    "bounds": [FloatVar(lb=0.0, ub=1.0) for _ in range(num_ops)],
    "minmax": "min",
    "log_to": None,
}

Número de operações: 4
Operações carregadas:
  0: Job job_1, Op 1, Máquinas [1, 2], Duração 1
  1: Job job_1, Op 2, Máquinas [1, 2], Duração 1
  2: Job job_2, Op 1, Máquinas [1, 2], Duração 1
  3: Job job_2, Op 2, Máquinas [1, 2], Duração 1

TESTE 1: Solução que RESPEITA a ordem das operações
Fitness (ordem correta): 2

TESTE 2: Solução que VIOLA a ordem das operações
⚠️  VIOLAÇÃO DE PRECEDÊNCIA: Job job_1 tentou executar Op 2, mas esperava Op 1
⚠️  VIOLAÇÃO DE PRECEDÊNCIA: Job job_2 tentou executar Op 2, mas esperava Op 1
🚨 SOLUÇÃO INVÁLIDA: 2 violações de precedência
   Makespan base: 2.00
   Penalidade: 2000
   Fitness final: 2002.00
Fitness (ordem violada): 2002

Diferença de fitness: 2000


## Simulated annealing

In [50]:
times = []
solutions = []
model = SA.OriginalSA(epoch=100)
for _ in range(30):
    start_time = time.time()
    g_best = model.solve(problem)
    end_time = time.time()
    times.append([end_time - start_time, g_best.id])
    solutions.append(g_best)




⚠️  VIOLAÇÃO DE PRECEDÊNCIA: Job job_1 tentou executar Op 2, mas esperava Op 1
🚨 SOLUÇÃO INVÁLIDA: 1 violações de precedência
   Makespan base: 3.00
   Penalidade: 1000
   Fitness final: 1003.00
⚠️  VIOLAÇÃO DE PRECEDÊNCIA: Job job_2 tentou executar Op 2, mas esperava Op 1
🚨 SOLUÇÃO INVÁLIDA: 1 violações de precedência
   Makespan base: 2.00
   Penalidade: 1000
   Fitness final: 1002.00
⚠️  VIOLAÇÃO DE PRECEDÊNCIA: Job job_2 tentou executar Op 2, mas esperava Op 1
🚨 SOLUÇÃO INVÁLIDA: 1 violações de precedência
   Makespan base: 2.00
   Penalidade: 1000
   Fitness final: 1002.00
⚠️  VIOLAÇÃO DE PRECEDÊNCIA: Job job_2 tentou executar Op 2, mas esperava Op 1
🚨 SOLUÇÃO INVÁLIDA: 1 violações de precedência
   Makespan base: 3.00
   Penalidade: 1000
   Fitness final: 1003.00
⚠️  VIOLAÇÃO DE PRECEDÊNCIA: Job job_2 tentou executar Op 2, mas esperava Op 1
🚨 SOLUÇÃO INVÁLIDA: 1 violações de precedência
   Makespan base: 2.00
   Penalidade: 1000
   Fitness final: 1002.00
⚠️  VIOLAÇÃO DE PRECEDÊNC

⚠️  VIOLAÇÃO DE PRECEDÊNCIA: Job job_2 tentou executar Op 2, mas esperava Op 1
🚨 SOLUÇÃO INVÁLIDA: 1 violações de precedência
   Makespan base: 2.00
   Penalidade: 1000
   Fitness final: 1002.00
⚠️  VIOLAÇÃO DE PRECEDÊNCIA: Job job_1 tentou executar Op 2, mas esperava Op 1
🚨 SOLUÇÃO INVÁLIDA: 1 violações de precedência
   Makespan base: 2.00
   Penalidade: 1000
   Fitness final: 1002.00
⚠️  VIOLAÇÃO DE PRECEDÊNCIA: Job job_1 tentou executar Op 2, mas esperava Op 1
🚨 SOLUÇÃO INVÁLIDA: 1 violações de precedência
   Makespan base: 2.00
   Penalidade: 1000
   Fitness final: 1002.00
⚠️  VIOLAÇÃO DE PRECEDÊNCIA: Job job_1 tentou executar Op 2, mas esperava Op 1
🚨 SOLUÇÃO INVÁLIDA: 1 violações de precedência
   Makespan base: 2.00
   Penalidade: 1000
   Fitness final: 1002.00
⚠️  VIOLAÇÃO DE PRECEDÊNCIA: Job job_1 tentou executar Op 2, mas esperava Op 1
🚨 SOLUÇÃO INVÁLIDA: 1 violações de precedência
   Makespan base: 2.00
   Penalidade: 1000
   Fitness final: 1002.00
⚠️  VIOLAÇÃO DE PRECEDÊNC

In [51]:

csv_filename = save_results_to_csv(times, solutions, data, "simulated_annealing_results.csv")


Resultados salvos em simulated_annealing_results.csv
Total de 30 soluções salvas
Timespan da instância: 2
